# Project Mobile Robot

(Prof. Francesco Mondada)

Students :  
Roxane Pangaud, Sélim Kamal, Yann Boudigou, Harsh Jaiswal

## Introduction

Our environment is an open space (white is preferred) on which we add black obstacle of random shapes. We founded more intresting to work with random obstacle than already define one (squares for exemple). 

We then use vision to transform the real map in a grid map. We chose to divide the space (150cm x 150cm) by 50x50 in squares, based on the pixels of an image taken by a smartphone. So in the end the space is divided in small squares of 3cm by 3cm.

We also use the vision to find the coordinates of the Thymio and the goal, as well as the angle of the robot.

After that, we use A* algorithm to find a path between the "start" (Thymio) and the "goal" (a target green and yellow).

Finally we use odometry and local avoidance in order for the Thymio to navigate in the environment. We sometimes check the position of the Thymio with the camera. And since we check in the camera sometimes, we can change the goal position if the robot isn't too close to finish it's run!

## Vision

*Yann ? Harsh?*

## Global Navigation

The first thing of our global navigation need is to get the occupancy_grid through our vision functions. This would gives a us a grid with all the cells that the robot cannnot be over because they are occupied or too close to an occupied cell for Thymio's width. 
And for this, the first step is to take a picture of the start situation.

In [ ]:
# Get a picture of the map 
img, useable = get_image()

When we do that, we need to make sure the camera wasn't obstructed and so we can have a valid grid. Otherwise our program won't work. Wether or not the camera was obstructed is in the useable boolean so we raise the following exception below

In [ ]:
if(not(useable)):
    raise Exception('Camera is obstructed')

Once this is done, if to Exception were raised, then we can proceed to get the actual occupany_grid which will be 50 by 50 according to the global variables that we set up before.

In [ ]:
occupancy_grid = get_occupancy_grid(img)

When this is done, we want to obtain the robot's position as well as the goal's position and then run an astar algorithm in order to find the best path between them. This path is finally given to our motion control module which will follow the given path while avoiding not-anticipated local obstacles. The astar algorithm will be implemented using the ressources given in exercise 5.

But in order to be able to detect a change of the goal's position that would mean the need of a new route and so a new application of the a star argument, we made sure our main motion control function(go_to_goal()) will return a true boolean if it achieved its purpose(reach the goal) or a false boolean if it has detected a change in the goal's position. And then we put this function in a while loop that makes sure everytime the go_to_goal function returns a false boolean, we get all the needed datas again and run a new astar algorithm with them. 

In [ ]:
#Set success as false as it hasn't reached the target at the beginning
success = False

#While the target is not reached
while(not(success)):
    # Get robot's and goal's new positions
    time.sleep(2)
    img, useable = get_image()
    if(not(useable)):
        raise Exception('Camera is obstructed')
    (robot_pos, start, goal) = get_data(img)
    # Find a new route
    path, visitedNodes = findroute(start, goal, occupancy_grid)
    # Plot everything concerning the route and astar algorithm execution
    (fig_astar, ax_astar) = plot_map(visitedNodes, path, start, goal, occupancy_grid)
    #Try to go to goal and see if goal's is reached(success=1) or if goal has moved(success=0)
    success = go_to_goal(path, occupancy_grid,  fig_astar, ax_astar)

## Motion Contol

For the motion control, we got inspired by the odometry done in the 6th exercise session. We first calibrate the Thymio, to understand what time it needed for turning 360 degrees. We tried to turn the Thymio and stoped at 340°, we then measured the time. With those data, we used rule of tree and tried to turn the Thymio in different angle. After some time tunning, it was good enough for our application. 

After that, we did the same for the robot's advance by measuring how much it advances for a certain amount of time.

Here are the global variable used for odometry and more generally the motion:

With this done, we created a function called "robot_evaluation" that need the position and the angle of the robot, as well as the position of the curent target and it returns *gamma*, the angle that the robot must turn to be aligned with the target in degrees, *direction* ("left","right" or "forward"), which indicate wich direction to turn and *distance*, the distance between the Thymio and the target (in cm). To calculate all of this, we use trigonometry for the different cases of relative positions between the Thymio and the target.

After that, we have all what we need to make the robot move. We implemented then 2 functions in order to move the Thymio: "robot_turn" and "robot_move_forward". The first one takes *gamma*, *direction* and the angle of the robot (*alpha*) and moves it in the direction and the angle (*gamma*) desired, then it changes *alpha*, regarding of how much it thinks the Thymio has moved (*gamma*). The direction changes also to "forward" since we now are aligned with the target. The second one makes the Thymio move forward. The function takes *distance* and the curent position of the robot and the target, and then move the robot forward by *distance* centimeters. Then it updates the position of the Thymio by saying that it is now on the target (target position = robot position).

With those displacement functions, we need to use local avoidance, in the case the Thymio meet an unexpected obstacle. This will be explained in the next chapiter. Here is the robot_turn funtion for you to see how it is implemented.

In [ ]:
def robot_turn(gamma, direction,alpha): #return alpha, direction, obstacle_seen (bool: 0->free)
    gamma = abs(gamma)
    time_turn = CALIBRATION_TIME_ANGLE*gamma/CALIBRATION_ANGLE
    obstacle_seen = 0
    
    if direction == "right":
        robot.set_var("motor.left.target", MOTOR_SPEED)
        robot.set_var("motor.right.target", int(int(2**16)-MOTOR_SPEED))
        t0 =time.time()
        while time.time()-t0 < time_turn :
            if (check_avoidance()) :
                obstacle_seen = 1
                avoid_obstacle()
                return alpha, direction, obstacle_seen
        
        alpha = angle_between_pi(alpha-math.radians(gamma))
        
    if direction == "left":
        robot.set_var("motor.left.target", int(int(2**16)-MOTOR_SPEED))
        robot.set_var("motor.right.target", MOTOR_SPEED)
        t0 =time.time()
        while time.time()-t0 < time_turn :
            if (check_avoidance()) :
                obstacle_seen = 1
                avoid_obstacle()
                return alpha, direction, obstacle_seen
        
        alpha = angle_between_pi(alpha+math.radians(gamma))
    
    direction = "forward"
    
    robot_stop()
    
    return alpha, direction, obstacle_seen

## Local navigation 

Local navigation is mainly about local avoidance. To do the local avoidance, we must first be familiar with the sensor. After some test on multiple surfaces, we come to a threshold number of 3800 (PROXIMITY_THRESHOLD) which correspond approximatively of 2cm.

So we created a function called "check_avoidance" that check if one of the sensors measures more than this threshold and returns 1 if it's the case. That means that there is something close to the front of the Thymio and we should go into the avoidance function.

One more specificity is that we don't consider the value of the sensors of the back (the 5th and 6th) because we use the cable to communicate with the robot.

For the avoidance specifically, we tried to implement 2 algorithm. The frist one was working with artificial neural network and the second one was working with conditions (basically: if, elseif, ...). We saw how the 2 algorithm were reacting and choose to continue with the conditional one, because it worked faster and was more reactive.

So after seeing an obstacle, the robot behaves like so:  
1) While the Thymio see the obstacle, it turns in the direction that requires the less rotation (it turns away from the side that see the most the obstacle)  
2) When it don't see it anymore, the robot continues to go forward a little bit to move away from the obstacle (DISTANCE_AVOIDANCE)  
3) Then, to avoid going back to the obstacle, we change the target by taing 8 (OFFSET_TARGET) target further in the id_target for the path of the A* list.  
4) Finaly, we get the position and the angle of the robot by the camera to update our code (we don't run odometry while avoiding an obstacle)

Here are the variables for the local avoidance, were TIME_SLEEP_OBSTACLE is used to add some time between 2 check in the point (1) :

The implementation of this algorithm in our code can be partially seen in the "robot_turn" function above. While the robot is turning or going forward, we constantly check if we see an obstacle with the "check_avoidance" function. If it sees one, a flag *obstacle_seen*, which is also an output of the function, turns to 1. And then we avoid the obstacle with the "avoid_obstacle" function that you see just below.

In [ ]:
def avoid_obstacle () :
    sensor_measurment = robot["prox.horizontal"]
    obstacle_direction = np.nonzero(sensor_measurment)[0]
    while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):

        obstacle_avoidance(obstacle_direction,sensor_measurment)

        time.sleep(TIME_SLEEP_OBSTACLE)

        sensor_measurment = robot["prox.horizontal"]
        obstacle_direction = np.nonzero(sensor_measurment)[0]
    
    robot_move_forward(DISTANCE_AVOIDANCE,0,0,0,0)
    robot_stop()
    
    return

This function measure the state of the Thymio's sensors and while the obstacle is seen, it sends it to "obstacle_avoidance", which is basically the conditionnal algorithm coupled with some communication functions to make the robot move.

Then we use the "robot_move_forward" function to make the robot move forward for DISTANCE_AVOIDANCE (15cm) and stop the robot, the time we get informations from the camera.

After all that, we return to the main function "go_to_goal" that set the offset for the target and get the informations from the camera to update the Thymio parameters. Here is a part of the "go_to_goal" function that we are interested in for local avoidance:

## All code and plotting

## Conclusion